In [18]:
import mongoengine
import meetup.api
import os
import configparser
import requests
import json
from opencage.geocoder import OpenCageGeocode

Configparser implementation

In [19]:
config = configparser.ConfigParser()
config.read('config/meetup.cfg')
os.environ['MEETUP_API_KEY'] = config['MEETUP']['API_KEY']
os.environ['OPENCAGE_KEY']=config['OPENCAGE']['KEY']

geocode = OpenCageGeocode(os.environ['OPENCAGE_KEY'])

In [3]:
class Event:
    
    def __init__(self,
                params,
                url_path):
        self.params = params
        self.url_path = url_path
        
    
    def get_event(self, *args, **kwargs):
        
        r = requests.get(self.url_path, params=self.params)
        request_string = r.text
        
        return request_string

In [4]:
default_args = dict(
    country='United Stated',
    key= os.environ['MEETUP_API_KEY'],
    topic='Python'
)

url_meetup_request = "https://api.meetup.com/2/open_events"
events = Event(params=default_args, url_path=url_meetup_request)
response = events.get_event()

In [9]:
json_response = json.loads(response)
data = json_response['results']
len(data)

191

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [7]:
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.5") \
    .getOrCreate()

In [89]:
columns = ['id', 'date', 'year', 'month', 'day', 'country', 'city', 'state', 'meetup_name', 'meetup_group_name', 'description', 'event_url', 'yes_rsvp_count', 'status']
id, date, year, month, day, country, address, meetup_name, meetup_group_name, description, event_url, yes_rsvp_count, status = ([] for i in range(13))

for label in data:
    date_event = datetime.fromtimestamp(label['time'] / 1000.0)
    location_json = geocode.reverse_geocode(label['group']['group_lat'], label['group']['group_lon'])
    location_meetup = location_json[0]['components']
    address_meetup = location_json[0]['formatted']
    
    
    id.append(label['id'])
    date.append(date_event)
    year.append(date_event.year)
    month.append(date_event.month)
    day.append(date_event.year)
    country.append(location_meetup['country'])
    address.append(address_meetup)
    meetup_name.append(label['name'])
    meetup_group_name.append(label['group']['name'])
    description.append(label['description'])
    event_url.append(label['event_url'])
    yes_rsvp_count.append(label['yes_rsvp_count'])
    status.append(label['status'])

In [64]:
# results_geocode = geocode.reverse_geocode(label['group']['group_lat'], label['group']['group_lon'])
    
results_geocode[0]['components']['city']

'Memphis'

In [88]:
location_json[0]['components']

{'ISO_3166-1_alpha-2': 'RO',
 'ISO_3166-1_alpha-3': 'ROU',
 '_type': 'building',
 'city': 'Iași',
 'continent': 'Europe',
 'country': 'Romania',
 'country_code': 'ro',
 'county': 'Iași',
 'house_number': '34',
 'political_union': 'European Union',
 'postcode': '507184',
 'residential': 'Iași',
 'road': 'Strada Florilor',
 'suburb': 'Gară'}